In [46]:
import cv2
import numpy as np
import math
from PIL import Image

In [47]:
global img_3,top_left,top_right,bottom_left,bottom_right
img_3 = np.zeros([720,1080,3],dtype=np.uint8)
img_3.fill(255)


In [48]:

def drawRectangle(action, x, y, flags, *userdata):
  # Referencing global variables 
  global img_3, top_left,top_right,bottom_left,bottom_right
  # Mark the top left corner when left mouse button is pressed
  if action == cv2.EVENT_LBUTTONDOWN:
    top_left_corner = [x,y]
    # When left mouse button is released, mark bottom right corner
  if action == cv2.EVENT_LBUTTONUP:
    bottom_right_corner = [x,y]
    # Draw the rectangle
    img_3 = np.zeros([720,1080,3],dtype=np.uint8)
    img_3.fill(255)
    cv2.rectangle(img_3, top_left_corner[0], bottom_right_corner[0], (0,0,0),thickness=2)
    top_left = [top_left_corner[0], top_left_corner[1]]
    top_right = [top_left_corner[0], bottom_right_corner[1]]
    bottom_left = [bottom_right_corner[0], top_left_corner[1]]
    bottom_right = [bottom_right_corner[0], bottom_right_corner[1]]
    cv2.circle(img_3, top_left, 5, (0,255,0),-1)
    cv2.circle(img_3, top_right, 5, (0,0,255),-1)
    cv2.circle(img_3, bottom_left, 5, (255,0,0),-1)
    cv2.circle(img_3, bottom_right, 5, (0,0,0),-1)
    cv2.imshow("3 Channel Window",img_3)


In [49]:
def cal_move_rect(first_transform, bottom_transform, top_left,top_right,bottom_left,bottom_right):
    vector = [(bottom_transform[0][0] - first_transform[0][0],bottom_transform[0][1]- first_transform[0][1])]
    tx=vector[0][0]
    ty=vector[0][1]
    T = np.array([[1,0,tx],[0,1,ty],[0,0,1]])
    top_left = np.array(top_left)
    top_right = np.array(top_right)
    bottom_left = np.array(bottom_left)
    bottom_right = np.array(bottom_right)
    rectangle = np.array([top_left,top_right,bottom_left,bottom_right])
    one = np.ones((rectangle.shape[0], 1))
    rectangle_matrix_ready = np.concatenate((rectangle,one), axis = 1)
    copyOfMatrixReady = rectangle_matrix_ready.copy()
    for i in range(rectangle_matrix_ready.shape[0]):
        copyOfMatrixReady[i] = T.dot(rectangle_matrix_ready[i])
    new_top_left = [(int(copyOfMatrixReady[0][0]), int(copyOfMatrixReady[0][1]))]
    top_left = new_top_left
    new_top_right = [(int(copyOfMatrixReady[1][0]), int(copyOfMatrixReady[1][1]))]
    top_right = new_top_right
    new_bottom_left = [(int(copyOfMatrixReady[2][0]), int(copyOfMatrixReady[2][1]))]
    bottom_left = new_bottom_left
    new_bottom_right = [(int(copyOfMatrixReady[3][0]), int(copyOfMatrixReady[3][1]))]
    bottom_right = new_bottom_right
    return top_left,top_right,bottom_left,bottom_right

In [50]:
def moveRectangle(action, x, y, flags, *userdata):
    global first_transform, bottom_transform
    global img_3, top_left,top_right,bottom_left,bottom_right
    if action == cv2.EVENT_LBUTTONDOWN:
        first_transform = [(x,y)]
        # When left mouse button is released, mark bottom right corner
    elif action == cv2.EVENT_LBUTTONUP:
        bottom_transform = [(x,y)]
        top_left,top_right,bottom_left,bottom_right = cal_move_rect(first_transform, bottom_transform, top_left,top_right,bottom_left,bottom_right)

        img_3 = np.zeros([720,1080,3],dtype=np.uint8)
        img_3.fill(255)
        # print('new_top_left_corner',top_left_corner)
        # print('new_bottom_right_corner',bottom_right_corner)
        img_3 = cv2.rectangle(img_3, top_left_corner[0], bottom_right_corner[0], (0,0,0),thickness=2)
        top_left = [top_left_corner[0][0], top_left_corner[0][1]]
        top_right = [top_left_corner[0][0], bottom_right_corner[0][1]]
        bottom_left = [bottom_right_corner[0][0], top_left_corner[0][1]]
        bottom_right = [bottom_right_corner[0][0], bottom_right_corner[0][1]]
        cv2.circle(img_3, top_left, 5, (0,255,0),-1)
        cv2.circle(img_3, top_right, 5, (0,0,255),-1)
        cv2.circle(img_3, bottom_left, 5, (255,0,0),-1)
        cv2.circle(img_3, bottom_right, 5, (0,0,0),-1)
        cv2.imshow("3 Channel Window",img_3)
        

In [51]:
def Calculate_degree(Center, Pointer):
    if Pointer[0][0] == Center[0][0]: # để tránh bị lỗi chia cho 0
        if Pointer[0][1] == Center[0][1]:
            return 0
        if Pointer[0][1] > Center[0][1]:
            return 90
        else:
            return 270
    degree = math.degrees(math.atan((Pointer[0][1] - Center[0][1])/(Pointer[0][0] - Center[0][0])))
    if degree < 0:
        degree += 360
    if Pointer[0][0] - Center[0][0] > 0: # x lon hon 0 
        if Pointer[0][1] - Center[0][1] > 0: # y lon hon 0 -> goc phan tu thu 1
            if degree > 90: 
                degree -= 180
        else: # goc phan tu thu 4
            if degree < 270:
                degree += 180
    else: # x nho hon 0
        if Pointer[0][1] - Center[0][1] > 0: # y lon hon 0 -> goc phan tu thu 2
            if degree > 180: 
                degree -= 180
        else: # goc phan tu thu 3
            if degree < 180:
                degree += 180 
    degree = 360 - degree
    return degree

In [52]:
def rotateRectangle(action, x, y, flags, *userdata):
    global Center, Pointer, img_3, top_left,top_right,bottom_left,bottom_right
    if action == cv2.EVENT_LBUTTONDOWN:
        Center = [(x,y)]
    elif action == cv2.EVENT_LBUTTONUP:
        height, width = img_3.shape[:2]
        Pointer = [(x,y)]
        degree = Calculate_degree(Center, Pointer)
        rotate_matrix = cv2.getRotationMatrix2D(center=Center[0], angle=degree, scale=1)
        # rectangle = np.array([top_left,top_right, bottom_left, bottom_right])
        # one = np.ones((rectangle.shape[0], 1))
        # rectangle_matrix_ready = np.concatenate((rectangle,one), axis = 1)
        # copyOfMatrixReady = rectangle_matrix_ready.copy()
        # print('rectangle_matrix_ready', rectangle_matrix_ready)
        # print('rotate_matrix',rotate_matrix) 
        # print(range(rectangle_matrix_ready.shape[0]))
        # for i in range(rectangle_matrix_ready.shape[0]):
        #     print('rotate_matrix',rotate_matrix.shape)
        #     print('copyOfMatrixReady', copyOfMatrixReady.shape)
        #     print('rectangle_matrix_ready', rectangle_matrix_ready[i].transpose().shape)
        #     copyOfMatrixReady[i] = np.dot(rotate_matrix,rectangle_matrix_ready[i].transpose())
        # cv2.line(img_3, rotate_matrix[0], rotate_matrix[1], (0,0,0), 2)
        # cv2.line(img_3, rotate_matrix[1], rotate_matrix[2], (0,0,0), 2)
        # cv2.line(img_3, rotate_matrix[2], rotate_matrix[3], (0,0,0), 2)
        # cv2.line(img_3, rotate_matrix[3], rotate_matrix[0], (0,0,0), 2)
        img_3 = cv2.warpAffine(src=img_3, M=rotate_matrix, dsize=(width, height))
        cv2.imshow("3 Channel Window", img_3)

In [53]:
def scaleRectangles(action, x, y, flags, *userdata):
    global scaleHead, scaleTail, img_3
    if action == cv2.EVENT_LBUTTONDOWN:
        scaleHead = [(x,y)]
    elif action == cv2.EVENT_LBUTTONUP:
        height, width = img_3.shape[:2]
        scaleTail = [(x,y)]
        if scaleTail[0][0] > scaleHead[0][0]:
            dist = (math.dist(list(scaleHead[0]), list(scaleTail[0]))/width)+ 1
        else:
            dist = 1- (math.dist(list(scaleHead[0]), list(scaleTail[0]))/width)
        rotate_matrix = cv2.getRotationMatrix2D(center=scaleHead[0], angle=0, scale=dist)
        img_3 = cv2.warpAffine(src=img_3, M=rotate_matrix, dsize=(width, height))
        cv2.imshow("3 Channel Window", img_3)

In [54]:
img_3 = np.zeros([720,1080,3],dtype=np.uint8)
img_3.fill(255)
# or img[:] = 255
# highgui function called when mouse events occur
cv2.namedWindow("3 Channel Window")
enable_draw_rectangle = False
enable_transform_rectangle = False
enable_rotate_rectangle = False
enable_scale_rectangle = False
while(1):
    cv2.imshow('3 Channel Window', img_3)



    k = cv2.waitKey(0)
    # if d is pressed, can draw
    if (k == 100): 
        enable_draw_rectangle = not enable_draw_rectangle
        if enable_draw_rectangle:
            cv2.setMouseCallback("3 Channel Window", drawRectangle)
        else:
            cv2.setMouseCallback("3 Channel Window", lambda *args : None)
    
    # if t is pressed, can transform
    if (k == 116): 
        pass
        enable_transform_rectangle = not enable_transform_rectangle
        if enable_transform_rectangle: 
            cv2.setMouseCallback("3 Channel Window", moveRectangle)
        else:
            cv2.setMouseCallback("3 Channel Window", lambda *args : None)

    # if r is pressed, can rotate
    if (k == 114):
        pass
        enable_rotate_rectangle = not enable_rotate_rectangle
        if enable_rotate_rectangle:
           cv2.setMouseCallback("3 Channel Window", rotateRectangle)
        else:
            cv2.setMouseCallback("3 Channel Window", lambda *args : None)

    # if s is pressed, can scale
    if (k == 115):
        pass
        enable_scale_rectangle = not enable_scale_rectangle
        if enable_scale_rectangle:
            cv2.setMouseCallback("3 Channel Window", scaleRectangles)
        else:
            cv2.setMouseCallback("3 Channel Window", lambda *args : None)

    # If c is pressed, out the image
    if (k == 99):
        img_3 = np.zeros([720,1080,3],dtype=np.uint8)
        img_3.fill(255)
        cv2.imshow('3 Channel Window', img_3)
    if (k == 113):
        break
cv2.destroyAllWindows()